In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

# Leemos nuestros dataframes
movies = pd.read_csv("./ml-latest-small/movies.csv", sep=",")
ratings = pd.read_csv("./ml-latest-small/ratings.csv", sep=",")
tags = pd.read_csv("./ml-latest-small/tags.csv", sep=",")
links = pd.read_csv("./ml-latest-small/links.csv", sep=",")

# Eliminamos nuelos
movies.dropna(inplace=True)
ratings.dropna(inplace=True)
tags.dropna(inplace=True)
links.dropna(inplace=True)

# Extraemos el año del título
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]
total=len(movies)
cuenta=0
nuestradatabase = pd.DataFrame()

def get_sinopsis(movie):
    titl = movies["title"]
    mov = movies.loc[titl==movie, "movieId"]
    movie_id = mov.iloc[0]
    movieId = links["movieId"]
    match = links.loc[movieId == movie_id, "tmdbId"]
    tmdb = match.iloc[0]
    tmdb = int(tmdb)
    tmdb = str(tmdb)
    moveiid = int(movie_id)
    moveiid = str(moveiid)
    url = "https://www.themoviedb.org/movie/" + tmdb
    #-----------------------
    chrome_options = Options()
    chrome_options.add_argument("--lang=en")
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(url)
    html = driver.page_source
    soup = bs(html)
    driver.quit()
    #-----------------------
    sinop = soup.find_all("div", class_="overview")
    for p in sinop:
        sinopsis = p.get_text(strip=True)
    #-----------------------
    rating = soup.find_all("span", class_="icon")
    for per in rating:
        valor = per.get('class')[1]
    match = re.search(r'icon-r\d\d', valor)
    if match:
        # Obtenemos el valor de los dos números
        porcentaje = match.group()[6:]
    global nuestradatabase
    nuestradatabase = nuestradatabase.append({'moveId':moveiid,'sinopsis':sinopsis,'rating':porcentaje}, ignore_index=True)

with ThreadPoolExecutor() as executor:
    results = [executor.submit(get_sinopsis, movie) for movie in movies["title"]]
    for f in as_completed(results):
        pass
nuestradatabase.to_csv('sinopsisDB.csv', index=False)

C:\Users\Carlow\AppData\Local\Temp\ipykernel_37732\810108244.py:48: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)
C:\Users\Carlow\AppData\Local\Temp\ipykernel_37732\810108244.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nuestradatabase = nuestradatabase.append({'moveId':moveiid,'sinopsis':sinopsis,'rating':porcentaje}, ignore_index=True)
C:\Users\Carlow\AppData\Local\Temp\ipykernel_37732\810108244.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nuestradatabase = nuestradatabase.append({'moveId':moveiid,'sinopsis':sinopsis,'rating':porcentaje}, ignore_index=True)
C:\Users\Carlow\AppData\Local\Temp\ipykernel_37732\810108244.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

In [ ]:
nuestradatabase